<a href="https://colab.research.google.com/github/narsym/Gender-detection-on-CelebA-dataset/blob/master/Gender_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing kaggle**

In [3]:
!pip install kaggle

**Uploading kaggle.json**

In [4]:
from google.colab import files
files.upload() #upload your kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"narasimha00076","key":"7c71718363fc16e0e7b623e90c58880a"}'}

Preparation

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Downloading dataset with api command

In [6]:
!kaggle datasets download -d jessicali9530/celeba-dataset

 99% 1.31G/1.33G [00:16<00:00, 84.5MB/s]
100% 1.33G/1.33G [00:16<00:00, 88.2MB/s]


Unzip the data

In [0]:
!unzip celeba-dataset.zip

Importing pandas and attributes of celebs 

In [1]:
import pandas as pd
data = pd.read_csv('list_attr_celeba.csv')
data.set_index('image_id',inplace = True)
data.head()

,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
image_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,1,-1,1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,1,-1,-1,1
000002.jpg,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1
000003.jpg,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1
000004.jpg,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,1,-1,1,-1,1,1,-1,1
000005.jpg,-1,1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1


getting male column from the data

In [0]:
gender = data.Male

Importing necessary libraries

In [0]:
import shutil
import glob
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

creating trash library, we dump, images which are already when training, here.

so when we read images for test, we do not get same images for test data

In [8]:
!mkdir trash

mkdir: cannot create directory ‘trash’: File exists


training and test lists

In [0]:
raw_train = []
raw_train_labels = []
raw_test = []
raw_test_labels = []

Get data

In [0]:
def get_images(data,no_of_images,labels):
  path = './img_align_celeba/img_align_celeba'
  files = os.listdir(path)
  mcount, fcount = 0, 0
  for i in files:
    if gender[i] == 1:
      mcount = mcount + 1
      if mcount == no_of_images:
        continue
    elif gender[i] == -1:
      gender[i] = 0
      fcount = fcount + 1
      if fcount == no_of_images:
        continue
    img = cv2.imread(os.path.join(path,i))
    shutil.move(os.path.join(path,i),'./trash')
    data.append(img)
    labels.append(main['Male'][i])
    if len(data) == 2 * no_of_images:
      return data, labels
    if len(data) % 100 == 0:
      print(len(data),'images')

Load data into raw_train and raw_test with labels for them

In [10]:
raw_train,raw_train_labels = get_images(raw_train,2500,raw_train_labels)
raw_test,raw_test_labels = get_images(raw_test,500,raw_test_labels)

100 images
200 images
300 images
400 images
500 images
600 images
700 images
800 images
900 images
1000 images
1100 images
1200 images
1300 images
1400 images
1500 images
1600 images
1700 images
1800 images
1900 images
2000 images
2100 images
2200 images
2300 images
2400 images
2500 images
2600 images
2700 images
2800 images
2900 images
3000 images
3100 images
3200 images
3300 images
3400 images
3500 images
3600 images
3700 images
3800 images
3900 images
4000 images
4100 images
4200 images
4300 images
4400 images
4500 images
4600 images
4700 images
4800 images
4900 images
100 images
200 images
300 images
400 images
500 images
600 images
700 images
800 images
900 images


again put images back into celeb images folder

In [0]:
files = os.listdir('./trash')
for i in files:
  shutil.move('./trash/' + i,'./img_align_celeba/img_align_celeba/' + i)

Check length of training and test data

In [13]:
len(raw_train),len(raw_train_labels),len(raw_test),len(raw_test_labels)

(5000, 5000, 1000, 1000)

save them for future use

In [0]:
np.save('raw_train.npy',raw_train)
np.save('raw_train_labels.npy',raw_train_labels)
np.save('raw_test.npy',raw_test)
np.save('raw_test_labels.npy',raw_test)

Converting data into dataset

In [0]:
train_data = tf.data.Dataset.from_tensor_slices((raw_train,raw_train_labels))
test_data = tf.data.Dataset.from_tensor_slices((raw_test,raw_test_labels))

Formatting images as per mobile net need

In [0]:
IMG_SIZE = 160
def format_example(image,label):
  image = tf.cast(image,dtype = tf.float32)
  image = (image / 255) - 1
  image = tf.image.resize(image,(IMG_SIZE,IMG_SIZE))
  return image, label
train_data = train_data.map(format_example)
test_data = test_data.map(format_example)

Shuffling data and converting them into batches

In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 2000
train_data = train_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_data = test_data.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

Mobile net V2 base model

In [17]:
IMG_SHAPE = (IMG_SIZE,IMG_SIZE,3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,include_top = False,weights = 'imagenet')
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

Checking image batch shape

In [34]:
for image_batch, label_batch in train_data.take(1):
  pass
print(image_batch.shape)

(32, 160, 160, 3)


checking output shape of base model

In [35]:
print(base_model(image_batch).shape)

(32, 5, 5, 1280)


creating global average layer and checking its output shape

In [36]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
print(global_average_layer(base_model(image_batch)).shape)

(32, 1280)


Creating prediction layer and checking its shape

In [37]:
prediction_layer = tf.keras.layers.Dense(1)
print(prediction_layer(global_average_layer(base_model(image_batch))).shape)

(32, 1)


Combining base model, global average layer and prediction layer into one model.

In [0]:
model = tf.keras.Sequential([
     base_model,
     global_average_layer,
     prediction_layer
])

Fixing learning rate and compiling with rmsprop optimizer

In [0]:
base_learning_rate = 0.0001
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = base_learning_rate),
              loss = 'binary_crossentropy', metrics = ['accuracy'])

summary of the model

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


Estimating the steps per epoch and validatiaon steps

In [41]:
num_train = 10000
num_test = 2500
initial_epochs = 20
steps_per_epochs = round(num_train) //  BATCH_SIZE
validation_steps = 4

loss0, accuracy0 = model.evaluate(test_data, steps = validation_steps)

4/4 [==============================] - 0s 15ms/step - loss: 6.1797 - accuracy: 0.3984


Training the model

In [42]:
history = model.fit(train_data,epochs = initial_epochs, validation_data = test_data)

Epoch 1/20
157/157 [==============================] - 7s 45ms/step - loss: 5.6114 - accuracy: 0.4174 - val_loss: 4.2084 - val_accuracy: 0.4720
Epoch 2/20
157/157 [==============================] - 7s 46ms/step - loss: 3.3787 - accuracy: 0.5144 - val_loss: 2.4536 - val_accuracy: 0.5940
Epoch 3/20
157/157 [==============================] - 7s 45ms/step - loss: 1.9852 - accuracy: 0.6738 - val_loss: 1.7931 - val_accuracy: 0.7270
Epoch 4/20
157/157 [==============================] - 7s 43ms/step - loss: 1.4984 - accuracy: 0.7588 - val_loss: 1.4151 - val_accuracy: 0.8010
Epoch 5/20
157/157 [==============================] - 7s 44ms/step - loss: 1.3091 - accuracy: 0.8102 - val_loss: 1.2384 - val_accuracy: 0.8220
Epoch 6/20
157/157 [==============================] - 7s 44ms/step - loss: 1.1991 - accuracy: 0.8290 - val_loss: 1.1892 - val_accuracy: 0.8290
Epoch 7/20
157/157 [==============================] - 7s 43ms/step - loss: 1.1225 - accuracy: 0.8398 - val_loss: 1.0701 - val_accuracy: 0.8450

Evaluating the model on test set

In [43]:
model.evaluate(test_data)

32/32 [==============================] - 1s 18ms/step - loss: 0.9176 - accuracy: 0.8860


[0.9176115989685059, 0.8859999775886536]

We get 90% accuracy on the validation set.